<a href="https://colab.research.google.com/github/Anjasfedo/Code-as-a-Cryptography/blob/main/eceg_points_after_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Parameters
p = 751  # prime modulus
a = -1    # coefficient of x in the elliptic curve
b = 188    # constant in the elliptic curve

In [2]:
import random
import sympy as sp
import json
import base64

class EllipticCurveElGamal:
    def __init__(self, a, b, p, k, B=None):
        self.p = p  # Prime modulus
        self.a = a  # Curve coefficient
        self.b = b  # Curve coefficient
        self.k = k  # Private key multiplier
        self.PointB = B  # Base point on the curve

        # Define mappings for characters
        self.char_to_num_dict = {
            '0': 0, '1': 1, '2': 2, '3': 3, '4': 4, '5': 5, '6': 6, '7': 7, '8': 8, '9': 9,
            'a': 10, 'b': 11, 'c': 12, 'd': 13, 'e': 14, 'f': 15, 'g': 16, 'h': 17, 'i': 18,
            'j': 19, 'k': 20, 'l': 21, 'm': 22, 'n': 23, 'o': 24, 'p': 25, 'q': 26, 'r': 27,
            's': 28, 't': 29, 'u': 30, 'v': 31, 'w': 32, 'x': 33, 'y': 34, 'z': 35, '.': 36,
            '#': 37, '*': 38, '/': 39, '-': 40
        }
        self.num_to_char_dict = {v: k for k, v in self.char_to_num_dict.items()}
        self.k_koblitz = max(self.char_to_num_dict.values()) + 1

    def elliptic_curve_equation(self, x):
        return (x**3 + self.a * x + self.b) % self.p

    def is_on_curve(self, x, y):
        return (y**2) % self.p == self.elliptic_curve_equation(x)

    def char_to_num(self, char):
        return self.char_to_num_dict[char]

    def num_to_char(self, num):
        return self.num_to_char_dict[num]

    def message_to_koblitz(self, message):
        return [self.char_to_num(char) for char in message]

    def koblitz_to_message(self, koblitz):
        return ''.join([self.num_to_char(num) for num in koblitz])

    def koblitz_encode(self, m, max_attempts=1000):
        num = 1
        attempts = 0
        while attempts < max_attempts:
            x = m * self.k_koblitz + num
            rhs = (x**3 + self.a * x + self.b) % self.p
            if sp.is_quad_residue(rhs, self.p):
                y = sp.sqrt_mod(rhs, self.p)
                return (x, y)
            num += 1
            attempts += 1
        raise ValueError(f"No valid point found for {m} after {max_attempts} attempts.")

    def koblitz_decode(self, x):
        return (x - 1) // self.k_koblitz

    def encrypt_message(self, message, public_key):
        koblitz_points = [self.koblitz_encode(m) for m in self.message_to_koblitz(message)]
        encrypted_points = []
        for M in koblitz_points:
            k = random.randint(1, self.p - 1)
            C1 = self.calculate_point_multiplication(self.PointB, k)
            C2 = self.calculate_point_addition(M, self.calculate_point_multiplication(public_key, k))
            encrypted_points.append((C1, C2))
        return encrypted_points

    def calculate_point_multiplication(self, P, k):
        if k == 0:
            return None
        result = None
        temp = P
        while k:
            if k & 1:
                result = self.calculate_point_addition(result, temp) if result else temp
            temp = self.calculate_point_addition(temp, temp)
            k >>= 1
        return result

    def calculate_point_addition(self, P, Q):
        if not P:
            return Q
        if not Q:
            return P
        x1, y1 = P
        x2, y2 = Q
        if x1 == x2 and y1 == y2:
            m = (3 * x1**2 + self.a) * pow(2 * y1, self.p - 2, self.p) % self.p
        else:
            m = (y2 - y1) * pow(x2 - x1, self.p - 2, self.p) % self.p
        x3 = (m**2 - x1 - x2) % self.p
        y3 = (m * (x1 - x3) - y1) % self.p
        return (x3, y3)

    def cipher_points_to_integers(self, cipher_points):
        return [
            ((C1[0] * self.p + C1[1]) % self.p, (C2[0] * self.p + C2[1]) % self.p)
            for C1, C2 in cipher_points
        ]

    def integers_to_cipher_points(self, cipher_integers):
        return [
            ((C1 // self.p, C1 % self.p), (C2 // self.p, C2 % self.p))
            for C1, C2 in cipher_integers
        ]

    def encode_integers_to_text(self, integers):
        charset = "abcdefghijklmnopqrstuvwxyz0123456789#*-/"
        base = len(charset)

        def to_base_string(number):
            if number == 0:
                return charset[0]
            encoded = []
            while number > 0:
                encoded.append(charset[number % base])
                number //= base
            return ''.join(reversed(encoded))

        return [
            (to_base_string(C1), to_base_string(C2)) for C1, C2 in integers
        ]

    def decode_text_to_integers(self, encoded_text):
        charset = "abcdefghijklmnopqrstuvwxyz0123456789#*-/"
        base = len(charset)

        def from_base_string(encoded):
            value = 0
            for char in encoded:
                value = value * base + charset.index(char)
            return value

        return [
            (from_base_string(C1), from_base_string(C2)) for C1, C2 in encoded_text
        ]


# Example Usage
if __name__ == "__main__":
    # Initialize curve parameters
    curve = EllipticCurveElGamal(a=2, b=3, p=751, k=1, B=(0, 1))

    # Generate private and public keys
    private_key = 15
    public_key = curve.calculate_point_multiplication(curve.PointB, private_key)

    # Example message
    message = "hello"

    # Encrypt the message to get cipher points
    encrypted_points = curve.encrypt_message(message, public_key)
    print("Cipher Points:", encrypted_points)

    # Convert cipher points to integers
    cipher_integers = curve.cipher_points_to_integers(encrypted_points)
    print("Cipher Integers:", cipher_integers)

    # Convert integers to ciphertext
    ciphertext = curve.encode_integers_to_text(cipher_integers)
    print("Ciphertext:", ciphertext)

    # Decode ciphertext back to integers
    decoded_integers = curve.decode_text_to_integers(ciphertext)
    print("Decoded Integers:", decoded_integers)

    # Convert integers back to cipher points
    decoded_points = curve.integers_to_cipher_points(decoded_integers)
    print("Decoded Cipher Points:", decoded_points)


Cipher Points: [((76, 715), (694, 650)), ((124, 641), (58, 155)), ((732, 705), (532, 95)), ((592, 571), (736, 544)), ((694, 328), (638, 214))]
Cipher Integers: [(715, 650), (641, 155), (705, 95), (571, 544), (328, 214)]
Ciphertext: [('r9', 'qk'), ('qb', 'd9'), ('rz', 'cp'), ('ol', 'ny'), ('ii', 'fo')]
Decoded Integers: [(715, 650), (641, 155), (705, 95), (571, 544), (328, 214)]
Decoded Cipher Points: [((0, 715), (0, 650)), ((0, 641), (0, 155)), ((0, 705), (0, 95)), ((0, 571), (0, 544)), ((0, 328), (0, 214))]
